In [171]:
import math, random
# a sequence of plot events in a coming of age story
# birth => misc. rite of passages => struggle ( + resolution ) => death
rite_of_passages = {
    'infancy': {  # life, firsts, parents
        'options':[ # events in chronological order
            'parents',
            'first steps'   
        ],
        'filler': ['playing', 'parents'] # events which can recur
    },
    'childhood': {  # kindergarten, learning, friends, parents
        'options':[ 
            'first pet',
            'first friend',
            'learning to cycle' 
        ],
        'filler': ['kindergarten', 'making friends']
    },
    'adolescence': { # highschool, parties, friends, romance
        'options':[ 
            'sixteenth birthday',
            'first relationship',
            'best friend',
            'prom',
            'moving out',
            'first drink',
            'learning to drive'
        ],
        'filler': ['party', 'friends', 'school']
    },
    'early_adult': { # independence, travel, graduation, early career
        'options':[ 
            'twenty first birthday',
            'graduation',
            'first job'
        ],
        'filler': ['party', 'work', 'university']
    },
    'mid_adult': { # settling down, career, marriage, children
        'options':[ 
            'marriage',
            'promotion',
            'children',
            'fiftieth birthday'
        ],
        'filler': ['work', 'friends']
    },
    'late_adult': { # retirement, grandchildren
        'options':[ 
            'retirement'
        ],
        'filler': ['grandchildren', 'hobby']
    }
}

struggles = [
    {
        'life_stage': 'late_adult',
        'name': 'sickness',
        'precondition': None
    },
    {
        'life_stage': 'adolescence',
        'name': 'parents_divorce',
        'precondition': None,
        
    },
    {
        'life_stage': 'mid_adult',
        'name': 'spouse_divorce',
        'precondition': 'marriage'
    },
    {
        'life_stage': 'late_adult',
        'name': 'spouse_death',
        'precondition': 'marriage'
    },
    {
        'life_stage': 'mid_adult',
        'name': 'friend_death',
        'precondition': None
    },
    {
        'life_stage': 'mid_adult',
        'name': 'child_death',
        'precondition': 'children'
    },
    {
        'life_stage': 'mid_adult',
        'name': 'redundancy',
        'precondition': 'career'
    }
]

class Plot: 
    def __init__(self, length):
        self.length = length if length > 20 else 20 # 3 events from each life stage minimum
        self.events = []
        self.struggle = random.choice(struggles)
        self.init_events()
            
    def init_events(self):
        class Event:
            def __init__(self, name=None, index=None, life_stage=None, image=None, text=None):
                self.name = name
                self.index = index
                self.life_stage = life_stage
                self.image = image
                self.text = text            
        length_of_struggle_stage = 0
        life_stages = list(rite_of_passages)
        life_stage_map = []
        # partition stages of life
        for i in range(self.length):
            index = i
            event = ''
            if index == 0:
                event = 'birth'
            elif index == self.length - 1:
                event = 'death'
            else:
                index = index - 1
                mapped_index = math.floor((index / (self.length - 2)) * len(life_stages))
                life_stage = life_stages[mapped_index]
                event = life_stage
            
            if event == self.struggle['life_stage']:
                length_of_struggle_stage = length_of_struggle_stage + 1
                
            self.events.append(event)
            
        life_stage_map = self.events.copy()
        # insert struggle
        min_start_offset = 1 if self.struggle['precondition'] is not None else 0
        start_of_struggle_stage = self.events.index(self.struggle['life_stage'])
        struggle_index = start_of_struggle_stage + length_of_struggle_stage - 1
        self.events[struggle_index] = self.struggle['name']
        
        # insert precondition
        if self.struggle['precondition'] is not None:
            self.events[start_of_struggle_stage] = self.struggle['precondition']

        # replace remaining life stages with rites of passage, making sure that the preconditions are always appended first
        rites = rite_of_passages
        for i in range(len(self.events)):
            value = self.events[i]
            if (value in life_stages):
                if len(rites[value]['options']) > 0:
                    event = rites[value]['options'].pop(0)
                    if (event in self.events):
                        event = rites[value]['options'].pop(0)
                else:
                    event = random.choice(rites[value]['filler'])
                self.events[i] = event
        
        # create event objects from events
        for i in range(len(self.events)):
            event = Event(
                name= self.events[i],
                index=i,
                life_stage = life_stage_map[i]
#                 image
#                 text
            )
            self.events[i] = event
            
k = Plot(20)
for z in k.events:
    print('Index: {}\nLife Stage: {}\nName: {}\nImage: {}\nText: {}\n==========='.format(z.index, z.life_stage, z.name, z.image, z.text))

Index: 0
Life Stage: birth
Name: birth
Image: None
Text: None
Index: 1
Life Stage: infancy
Name: parents
Image: None
Text: None
Index: 2
Life Stage: infancy
Name: first steps
Image: None
Text: None
Index: 3
Life Stage: infancy
Name: playing
Image: None
Text: None
Index: 4
Life Stage: childhood
Name: first pet
Image: None
Text: None
Index: 5
Life Stage: childhood
Name: first friend
Image: None
Text: None
Index: 6
Life Stage: childhood
Name: learning to cycle
Image: None
Text: None
Index: 7
Life Stage: adolescence
Name: sixteenth birthday
Image: None
Text: None
Index: 8
Life Stage: adolescence
Name: first relationship
Image: None
Text: None
Index: 9
Life Stage: adolescence
Name: best friend
Image: None
Text: None
Index: 10
Life Stage: early_adult
Name: twenty first birthday
Image: None
Text: None
Index: 11
Life Stage: early_adult
Name: graduation
Image: None
Text: None
Index: 12
Life Stage: early_adult
Name: first job
Image: None
Text: None
Index: 13
Life Stage: mid_adult
Name: children
